In [1]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(course_api_key=course_api_key)

In [8]:
import pandas as pd
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from tqdm import tqdm

df = pd.read_csv('https://stepik.org/media/attachments/lesson/1110806/vacancies_messages_50.csv')


In [9]:
template = """Из следующего текста извлеки информацию:

job_title: описании вакансии, на том же языке. Проверяем полное совпадение, без учета регистра.Без грейда

company: название компании без учета регистра

salary: исла пишем без пробелов, не пишем тыс. или к.(умножаем в таком случае на 1000), не пишем фикс, плюшки, премии, % от продаж, техника

tg: Указываем контакт в телеграмм, используя @. Проверяем полное совпадение, с учетом регистра.
Если указано несколько контаков, то указываем их через запятую (не забывая про пробел после запятой).

grade: Возможные значения intern, junior, junior+, middle, middle+, senior, lead.
Если указано несколько значений, то пишем их через запятую в порядке возрастания.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)

job_title_schema = ResponseSchema(
    name="job_title",
    description="..."
)


company_schema = ResponseSchema(
    name="company",
    description="..."
)


salary_schema = ResponseSchema(
    name="salary",
    description="..."
)


tg_schema = ResponseSchema(
    name="tg",
    description="..."
)


grade_schema = ResponseSchema(
    name="grade",
    description="..."
)

response_schemas = [
    job_title_schema,
    company_schema,
    salary_schema,
    tg_schema,
    grade_schema
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas) # Создаём парсер и подаём в него список со схемами
format_instructions = output_parser.get_format_instructions() # Получаем инструкции по форматированию ответа

dict_list = [] # Список из словарей, где будем хранить полученную информацию
for text_input in tqdm(df['text']):
    messages = prompt.format_messages(
        text=text_input,
        format_instructions=format_instructions
    )
    response = llm.invoke(messages) # Ответ модели
    dict_list.append(output_parser.parse(response.content)) # Переводим содержимое ответа модели в словарь и добавляем в список
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает
    
result_df = pd.DataFrame(dict_list) # Создаем датафрейм из полученной информации
ans_df = pd.concat([df, result_df], axis=1) # Объединяем всё в один датафрейм
ans_df.to_csv('2_2_8_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)    

100%|██████████| 50/50 [01:15<00:00,  1.51s/it]
